In [1]:
!pip install openai &> /dev/null
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [2]:
import gradio as gr
import openai

openai.api_key = "EMPTY"
openai.api_base = "http://zanino.millennium.berkeley.edu:8000/v1"

def get_gorilla_response(prompt, model="gorilla-7b-hf-v1"):
  completion = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "user", "content": prompt}]
  )
  return completion.choices[0].message.content

def parse_output(text):
  components = {}

  components['domain'] = text.split("<<<domain>>>:")[1].split("<<<api_call>>>")[0].strip()
  components['api_call'] = text.split("<<<api_call>>>:")[1].split("<<<api_provider>>>")[0].strip()
  components['api_provider'] = text.split("<<<api_provider>>>:")[1].split("<<<explanation>>>")[0].strip()
  components['explanation'] = text.split("<<<explanation>>>:")[1].split("<<<code>>>")[0].strip()
  components['code'] = text.split("<<<code>>>:")[1].strip()

  return components

def parse_and_display(prompt):

  text = get_gorilla_response(prompt)

  components = parse_output(text)

  domain = components['domain']
  api_call = components['api_call']
  api_provider = components['api_provider']
  explanation = components['explanation']
  code = components['code']

  return domain, api_call, api_provider, explanation, code

examples = [
  ["Translate this sentence to French"],
  ["Summarize this paragraph in 5 sentences"],
  ["Generate an image of a cat"]
]

iface = gr.Interface(fn=parse_and_display,
                     inputs="text",
                     outputs=[
                       gr.components.Textbox(label="Domain"),
                       gr.components.Textbox(label="API Call"),
                       gr.components.Textbox(label="API Provider"),
                       gr.components.Textbox(label="Explanation"),
                       gr.components.Code(label="Code")
                     ],
                     title="Gorilla API Explorer",
                     description="A simple user friendly UI to interact with Gorilla API",
                     examples=examples)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://25e20add22e809b34b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
